In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from tqdm import tqdm

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from datasets import load_metric
from torch.utils.data import DataLoader
import random

In [ ]:
s2+VS+CNN & s2+PB+LSTM & 0.8398          & 0.8222          & 0.8305          & 0.3977          & 0.7157          & 0.5116         & 0.5779          \\
s2+VS+CNN & s3+PB+LSTM & 0.8398          & 0.8222          & 0.8305          & 0.3978          & 0.7157          & 0.5117         & 0.5779          \\
s3+VS+CNN & s2+PB+LSTM & 0.8395          & 0.8223          & 0.8304          & 0.3982          & 0.7146          & 0.5108         & 0.5778          \\
s3+VS+CNN & s3+PB+LSTM & 0.8395          & 0.8223          & 0.8304          & 0.3983          & 0.7146          & 0.5109         & 0.5779          \\

s1+vs+lstm+1 & s2+distil+cnn+1
s1+vs+lstm+5 & s2+viso+lstm+5
s1+vs+lstm+1 & s2+viso+lstm+5
s1+vs+lstm+5 & s2+distil+cnn+1

In [2]:
def generate_task_3(model, tokenizer, dataloader, target_len=512, device='cpu'):

    predictions = []
    references = []

    model.eval()
    with torch.no_grad():
        # Wrap the dataloader with tqdm to monitor progress
        with tqdm(dataloader, desc="Evaluation") as tqdm_loader:
            for batch_idx, data in enumerate(tqdm_loader, 0):
                tqdm_loader.set_description(f"Validation, Batch {batch_idx + 1}/{len(dataloader)}")

                ids = data['input']['input_ids'].to(device, dtype=torch.long)
                mask = data['input']['attention_mask'].to(device, dtype=torch.long)
                y = data['label']['input_ids'].to(device, dtype=torch.long)

                generated_ids = model.generate(
                    input_ids=ids,
                    attention_mask=mask,
                    max_length=target_len,
                    num_beams=2,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True
                )

                preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
                target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]

                predictions.extend(preds)
                references.extend(target)
                
    return predictions, references

def compute_score_task_3(predictions, references):
    bertscore_metric = load_metric('bertscore')
    bleu_metric = load_metric('bleu')
    rouge_metric = load_metric('rouge')

    bertscore_result = bertscore_metric.compute(predictions=predictions, references=references, lang="vi")
    bertscore_precision = round(np.mean(bertscore_result['precision']), 4)
    bertscore_recall = round(np.mean(bertscore_result['recall']), 4)
    bertscore_f1 = round(np.mean(bertscore_result['f1']), 4)

    bleuscore_result = bleu_metric.compute(
        predictions=[pred.split() for pred in predictions],
        references=[[ref.split()] for ref in references],
    )
    bleuscore = round(bleuscore_result['bleu'], 4)
    bleu_prec_1 = round(bleuscore_result['precisions'][0])
    bleu_prec_2 = round(bleuscore_result['precisions'][1])
    bleu_prec_3 = round(bleuscore_result['precisions'][2])
    bleu_prec_4 = round(bleuscore_result['precisions'][3])

    rouge_result = rouge_metric.compute(predictions=predictions, references=references)
    rouge_1 = round(rouge_result['rouge1'].mid.fmeasure, 4)
    rouge_2 = round(rouge_result['rouge2'].mid.fmeasure, 4)
    rouge_L = round(rouge_result['rougeL'].mid.fmeasure, 4)
    
    return (bertscore_precision, bertscore_recall, bertscore_f1), \
           (bleuscore, bleu_prec_1, bleu_prec_2, bleu_prec_3, bleu_prec_4), \
           (rouge_1, rouge_2, rouge_L)

In [3]:
df = pd.read_csv(r"E:\Learning\Docker_basic\basic_kafka\kltn\experiments\results\task3\prediction\outputs_lstm-viso-1_cnn-distil-1_bartpho-syllable_test.csv", index_col=0)

In [4]:
###### Model

# "vinai/bartpho-syllable-base"
# "vinai/bartpho-word-base"
# "VietAI/vit5-base"

model = "vinai/bartpho-syllable-base"

tokenizer = AutoTokenizer.from_pretrained(model)
generation_model = AutoModelForSeq2SeqLM.from_pretrained(model)
# generation_model = generation_model.to(device='gpu')
# generation_model.load_state_dict(torch.load(r"E:\Learning\Docker_basic\basic_kafka\kltn\experiments\results\task3\vit5-base_2.pth"))

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--vinai--bartpho-syllable-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ImportError: 
BartphoTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
